**NYPD Patrol Assistance System**

In [2]:

import pandas as pd
from pyspark import SparkContext 
from pyspark.sql import SparkSession
from pyspark.sql import functions as fn
from pyspark.ml import feature
from pyspark.ml import classification
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import StringIndexer
from pyspark.mllib.evaluation import BinaryClassificationMetrics as metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

**Data Source **

The data can be obtained from the following Website:

data.ny.gov in the name of "Crash data"


**Versions of Software**


The version of the software is Python 3.0 and Apache Spark 2.2.0

In [4]:
$$$$$$$$$$$$$$$$$$$$$$$$$$$$#Test data with only one row to check prediction
test_data = sqlContext.read.format("csv").option("header", "true").load("/FileStore/tables/data1.csv")

In [5]:
#Loading data into databricks as a spark dataframe
myData1 = sqlContext.read.format("csv").option("header", "true").load("/FileStore/tables/Category.csv")

In [6]:
#Loading data onto databricks as a spark dataframe
myData = sqlContext.read.format("csv").option("header", "true").load("/FileStore/tables/myData.csv")

In [7]:
#Displaying data as a table
display(myData)

Year Crash Descriptor Time Date Day of Week Police Report Lighting Conditions Municipality Collision Type Descriptor County Name Road Descriptor Weather Conditions Traffic Control Device Road Surface Conditions DOT Reference Marker Location Pedestrian Bicyclist Action Event Descriptor Number of Vehicles Involved 2014 Property Damage Accident 8:30 1/3/2014 Friday Y Daylight WATERTOWN OVERTAKING JEFFERSON Straight and Level Clear None Snow/Ice null Not Applicable Other Motor Vehicle, Collision With 2 2014 Property Damage & Injury Accident 14:48 1/3/2014 Friday Y Daylight WATERTOWN REAR END JEFFERSON Straight and Level Cloudy Traffic Signal Snow/Ice 3 73022024 Not Applicable Other Motor Vehicle, Collision With 2 2014 Property Damage & Injury Accident 11:30 1/4/2014 Saturday Y Daylight WATERTOWN OTHER JEFFERSON Straight and Grade Clear None Snow/Ice null Not Applicable Building/Wall, Collision With Fixed Object 1 2014 Property Damage Accident 7:45 1/3/2014 Friday Y Daylight WATERTOWN RIGHT ANGLE JEFFERSON Straight and Level Clear Stop Sign Snow/Ice null Not Applicable Other Motor Vehicle, Collision With 2 2014 Property Damage Accident 15:11 1/4/2014 Saturday Y Daylight WATERTOWN RIGHT ANGLE JEFFERSON Straight and Level Cloudy None Wet 3 73022020 Not Applicable Other Motor Vehicle, Collision With 2 2014 Property Damage Accident 17:12 1/4/2014 Saturday Y Dark-Road Unlighted FRANKFORT OTHER HERKIMER Straight and Level Cloudy None Snow/Ice null Not Applicable Snow Embankment, Collision With Fixed Object 1 2014 Injury Accident 15:15 1/3/2014 Friday Y Daylight WARSAW REAR END WYOMING Straight and Level Clear None Snow/Ice 19 46021177 Not Applicable Other Motor Vehicle, Collision With 2 2014 Property Damage Accident 11:30 1/3/2014 Friday Y Daylight ROTTERDAM OTHER SCHENECTADY Curve and Level Snow None Snow/Ice 890I16013001 Not Applicable Median - Not At End, Collision With Fixed Object 1 2014 Property Damage Accident 15:30 1/3/2014 Friday Y Daylight EAST AURORA RIGHT ANGLE ERIE Straight and Level Clear None Dry 16 53021191 Not Applicable Other Motor Vehicle, Collision With 2 2014 Property Damage & Injury Accident 8:02 1/3/2014 Friday Y Daylight AURORA OTHER ERIE Curve and Grade Cloudy None Snow/Ice 400 53011024 Not Applicable Guide Rail - Not At End, Collision With Fixed Object 1 2014 Property Damage & Injury Accident 16:20 1/3/2014 Friday Y Daylight AMHERST REAR END ERIE Straight and Level Clear None Dry null Not Applicable Other Motor Vehicle, Collision With 2 2014 Property Damage Accident 14:23 1/4/2014 Saturday Y Daylight AMHERST OVERTAKING ERIE Curve at Hill Crest Clear None Wet 952T53011003 Not Applicable Other Motor Vehicle, Collision With 2 2014 Property Damage & Injury Accident 10:28 1/3/2014 Friday Y Daylight AMHERST OVERTAKING ERIE Straight and Grade Clear None Snow/Ice 290I53011078 Not Applicable Other Motor Vehicle, Collision With 2 2014 Property Damage Accident 8:03 1/3/2014 Friday Y Daylight AMHERST OVERTAKING ERIE Straight and Grade Clear None Snow/Ice 290I53011074 Not Applicable Other Motor Vehicle, Collision With 2 2014 Property Damage Accident 10:38 1/3/2014 Friday Y Daylight AMHERST OTHER ERIE Straight and Grade Clear None Snow/Ice 290I53011076 Not Applicable Guide Rail - Not At End, Collision With Fixed Object 1 2014 Property Damage Accident 16:31 1/3/2014 Friday Y Daylight AMHERST REAR END ERIE Straight and Level Clear Traffic Signal Wet null Not Applicable Other Motor Vehicle, Collision With 2 2014 Property Damage Accident 15:29 1/1/2014 Wednesday Y Daylight AMHERST OTHER ERIE Curve and Grade Cloudy None Dry 290I53011072 Not Applicable Guide Rail - Not At End, Collision With Fixed Object 1 2014 Property Damage Accident 15:55 1/3/2014 Friday Y Daylight TONAWANDA OTHER ERIE Straight and Level Clear None Snow/Ice 290I53011009 Not Applicable Other Motor Vehicle, Collision With 3 2014 Property Damage Accident 9:07 1/3/2014 Friday Y Daylight TONAWANDA OTHER ERIE Curve and Grade Clear None Snow/Ice 290I53011042 Not Applic

**Displaying the dimension of the data**

In [9]:
#print((myData.count(), len(df.columns)))
print 'Number of rows in the dataframe : ', myData.count()
print 'Number of columns in the dataframe :' , len(myData.columns)

Number of rows in the dataframe : 895916
Number of columns in the dataframe : 18

In [10]:
#Displaying t the column names
myData.columns

Out[ 8 ]: 
['Year',
 'Crash Descriptor',
 'Time',
 'Date',
 'Day of Week',
 'Police Report',
 'Lighting Conditions',
 'Municipality',
 'Collision Type Descriptor',
 'County Name',
 'Road Descriptor',
 'Weather Conditions',
 'Traffic Control Device',
 'Road Surface Conditions',
 'DOT Reference Marker Location',
 'Pedestrian Bicyclist Action',
 'Event Descriptor',
 'Number of Vehicles Involved']

In [11]:
type(myData)

Out[ 10 ]: pyspark.sql.dataframe.DataFrame

In [12]:
#Converting the dataframe to a pandas dataframe to manipulate the variables
Data_Pandas = myData.toPandas()


In [13]:
#Isolating the target column
Predicted =Data_Pandas['Crash Descriptor']

In [14]:
#Extracting the time column from the dataframe
Time = Data_Pandas['Time']
#Extracting hour information from time
Time = Time.map(lambda x:x.split(':'))
Time = Time.map(lambda x: x[0])

In [15]:
#Extracted the hour alone after eliminating the minutes
Time

Out[ 17 ]: 
0 8
1 14
2 11
3 7
4 15
5 17
6 15
7 11
8 15
9 8
10 16
11 14
12 10
13 8
14 10
15 16
16 15
17 15
18 9
19 17
20 14
21 9
22 13
23 20
24 18
25 13
26 16
27 20
28 21
29 16
 ..
895886 9
895887 13
895888 8
895889 8
895890 16
895891 14
895892 6
895893 22
895894 13
895895 21
895896 21
895897 0
895898 12
895899 0
895900 9
895901 9
895902 1
895903 9
895904 6
895905 15
895906 12
895907 1
895908 13
895909 16
895910 1
895911 15
895912 15
895913 19
895914 14
895915 15
Name: Time, dtype: object

In [16]:
#Extracting date column from the dataframe
Date = Data_Pandas['Date']

In [17]:
#Extracting month information from date
Date = Date.map(lambda x:x.split('/'))
Date  = Date.map(lambda x:x[0])

In [18]:
#Deleting date and time column from original dataframe to avoid problems while merging
del Data_Pandas['Date']
del Data_Pandas['Time']

In [19]:
#Displaying columns without date and time
Data_Pandas.columns

Out[ 21 ]: 
Index([u'Year', u'Crash Descriptor', u'Day of Week', u'Police Report',
 u'Lighting Conditions', u'Municipality', u'Collision Type Descriptor',
 u'County Name', u'Road Descriptor', u'Weather Conditions',
 u'Traffic Control Device', u'Road Surface Conditions',
 u'DOT Reference Marker Location', u'Pedestrian Bicyclist Action',
 u'Event Descriptor', u'Number of Vehicles Involved'],
 dtype='object')

In [20]:
#Joining the new date and time columns  to the dataframe
Data_Pandas = Data_Pandas.join(Time,how = "outer")
Data_Pandas = Data_Pandas.join(Date,how = "outer")

In [21]:
#Validating the join by checking a random row value
Data_Pandas.iloc[818078]

Out[ 48 ]: 
Year 2016
Crash Descriptor Property Damage Accident
Day of Week Monday
Police Report Y
Lighting Conditions Daylight
Municipality HEMPSTEAD
Collision Type Descriptor OTHER
County Name NASSAU
Road Descriptor Straight and Level
Weather Conditions Clear
Traffic Control Device None
Road Surface Conditions Dry
DOT Reference Marker Location None
Pedestrian Bicyclist Action Not Applicable
Event Descriptor Other Motor Vehicle, Collision With
Number of Vehicles Involved 6
Time 16
Month 10
Fatal Accident N
Name: 818078, dtype: object

In [22]:
#Display columns for validation
Data_Pandas.columns

Out[ 33 ]: 
Index([u'Year', u'Crash Descriptor', u'Day of Week', u'Police Report',
 u'Lighting Conditions', u'Municipality', u'Collision Type Descriptor',
 u'County Name', u'Road Descriptor', u'Weather Conditions',
 u'Traffic Control Device', u'Road Surface Conditions',
 u'DOT Reference Marker Location', u'Pedestrian Bicyclist Action',
 u'Event Descriptor', u'Number of Vehicles Involved', u'Time', u'Date'],
 dtype='object')

In [23]:
#Creating the Predicted variable Fatal Accident from Crash descriptor variable
#If the Cr
Data_Pandas['Fatal Accident'] = Data_Pandas['Crash Descriptor'].apply(lambda x: 'Y' if x == 'Fatal Accident' else 'N')

In [24]:
#Validating the join
Data_Pandas.columns

Out[ 44 ]: 
Index([u'Year', u'Crash Descriptor', u'Day of Week', u'Police Report',
 u'Lighting Conditions', u'Municipality', u'Collision Type Descriptor',
 u'County Name', u'Road Descriptor', u'Weather Conditions',
 u'Traffic Control Device', u'Road Surface Conditions',
 u'DOT Reference Marker Location', u'Pedestrian Bicyclist Action',
 u'Event Descriptor', u'Number of Vehicles Involved', u'Time', u'Date',
 u'Fatal Accident'],
 dtype='object')

In [25]:
#Renaming the date column as month
Data_Pandas = Data_Pandas.rename(columns = {'Date':'Month'})

In [26]:
#Checking for NULL values in the predicted variable
Data_Pandas['Fatal Accident'].isnull().values.any()

Out[ 46 ]: False

In [27]:
Data_Pandas['Fatal Accident'].describe(include = 'all')

Out[ 47 ]: 
count 895916
unique 2
top N
freq 892936
Name: Fatal Accident, dtype: object

In [28]:
#Converting the pandas dataframe to a spark dataframe
Data = sqlContext.createDataFrame(Data_Pandas)


In [29]:
#Displaying the spark dataframe
Data.columns

Out[ 50 ]: 
['Year',
 'Crash Descriptor',
 'Day of Week',
 'Police Report',
 'Lighting Conditions',
 'Municipality',
 'Collision Type Descriptor',
 'County Name',
 'Road Descriptor',
 'Weather Conditions',
 'Traffic Control Device',
 'Road Surface Conditions',
 'DOT Reference Marker Location',
 'Pedestrian Bicyclist Action',
 'Event Descriptor',
 'Number of Vehicles Involved',
 'Time',
 'Month',
 'Fatal Accident']

**Data Splitting**

The data is split into three parts:

**Training**: To fit the model

**Validation**: For Model Selection

**Testing**:For Testing the Model

In [31]:
#Splitting the data into test and train
training,validation,test = Data.randomSplit([0.7, 0.2,0.1], 0)

In [32]:
#Bucketizing time into various bins
splits = [0,2,4,6,8,10,12,14,16,18,20,22,24]

MODEL 1: **Logistic Regression** (with elastic net Regularization)

The feature Engineering techniques used are:

**StringIndexer**

**OneHotEncoder**

**Bucketizer**(For Time)

All these techniques are pipelined and the Machine Learning models are fitted with the training data and validated on validation data

In [34]:
#Model 1 - Logistic Regression
model1 = Pipeline(stages=[feature.StringIndexer(inputCol='Month', outputCol='encoded_Month'),
                          feature.StringIndexer(inputCol = 'County Name',outputCol = 'encoded_County'),
                          feature.StringIndexer(inputCol = 'Day of Week',outputCol = 'encoded_Day'),
                          feature.StringIndexer(inputCol = 'Fatal Accident',outputCol = 'encoded_Fatalaccident'),
                          feature.StringIndexer(inputCol = 'Time',outputCol = 'encoded_Time'),
                          feature.OneHotEncoder(inputCol = 'encoded_Month',outputCol = 'final_Month'),
                          feature.OneHotEncoder(inputCol = 'encoded_Day',outputCol = 'final_Day'),
                          feature.OneHotEncoder(inputCol = 'encoded_County',outputCol = 'final_County'),
                          feature.Bucketizer(splits=splits, inputCol='encoded_Time', outputCol="bucketed_Time"),
  feature.VectorAssembler(inputCols = ['final_Month','final_Day','bucketed_Time','final_County'],outputCol = 'final_features'),
 classification.LogisticRegression(labelCol='encoded_Fatalaccident', featuresCol='final_features',regParam = 0.1,elasticNetParam = 0.002)])

In [35]:
#Fitting model1 with training data
fit1 = model1.fit(training)

In [36]:
#Transforming model0 on to validation data
dataframe1 = fit1.transform(validation)

In [37]:
#Prediction vs Actual for model0
df1 = dataframe1.select('Prediction','Fatal Accident')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4345105894762172> in <module> () 
 1 #Prediction vs Actual for model0 
 ----> 2 df1 = dataframe1 . select ( 'Prediction' , 'Fatal Accident' ) 

 NameError : name 'dataframe1' is not defined

In [38]:
# Print the coefficients and intercept for logistic regression without regularization
lrm = fit1.stages[-1]
lrm.coefficients 

Out[ 164 ]: DenseVector([-0.1427, -0.075, -0.0479, 0.0158, 0.0698, 0.0931, -0.1623, 0.0585, 0.0877, 0.1082, -0.089, -0.0128, -0.006, -0.0045, -0.0002, -0.0063, 0.0064, -0.0164, -0.0847, -0.0171, 0.7979, 0.981, -0.0017, -0.052, -0.2295, 0.7168, 0.9733, -0.2118, -0.3229, -0.3291, -0.1618, -0.1899, -0.4311, -0.3285, -0.341, -0.3271, 0.619, -0.2311, -0.3409, -0.377, -0.3567, -0.4291, -0.301, -0.5799, -0.5312, -0.368, -0.3489, -0.5941, -0.4747, -0.3527, -0.4535, -0.5233, -0.6192, -0.3293, -0.4219, -0.3206, -0.6218, -0.3321, -0.4344, -0.5129, -0.4755, -0.4933, -0.4621, -0.638, -0.6109, -0.4053, -0.5256, -0.2752, -0.4476, -0.6733, -0.2533, -0.4103, -0.3341, -0.4684, -0.412, -0.352, -0.4445, -0.6312, -0.5329, -0.6073])

In [39]:
# Print the coefficients and intercept for logistic regression with regularization
#There is a decrease in the magnitude of coefficients
lrm = fit1.stages[-1]
lrm.coefficients

Accuracy is calculated using Multiclass Classification Evaluator and it is found to be 67%

In [41]:
#Accuracy
evaluator = MulticlassClassificationEvaluator(labelCol = 'encoded_Fatalaccident', predictionCol = 'prediction',metricName = 'accuracy')
accuracy1 = evaluator.evaluate(dataframe1)
#Error percentage:
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.333693

MODEL 2: **Random Forest**

The feature Engineering techniques used are:

**StringIndexer**

**OneHotEncoder**

**Bucketizer**(For Time)

All these techniques are pipelined and the Machine Learning models are fitted with the training data and validated on validation data

In [43]:
#Model 2
model1 = Pipeline(stages=[feature.StringIndexer(inputCol='Month', outputCol='encoded_Month'),
                          feature.StringIndexer(inputCol = 'County Name',outputCol = 'encoded_County'),
                          feature.StringIndexer(inputCol = 'Day of Week',outputCol = 'encoded_Day'),
                          feature.StringIndexer(inputCol = 'Fatal Accident',outputCol = 'encoded_Fatalaccident'),
                          feature.StringIndexer(inputCol = 'Time',outputCol = 'encoded_Time'),
                          feature.OneHotEncoder(inputCol = 'encoded_Month',outputCol = 'final_Month'),
                          feature.OneHotEncoder(inputCol = 'encoded_Day',outputCol = 'final_Day'),
                          feature.OneHotEncoder(inputCol = 'encoded_County',outputCol = 'final_County'),
                          feature.Bucketizer(splits=splits, inputCol='encoded_Time', outputCol="bucketed_Time"),
                 feature.VectorAssembler(inputCols = ['final_Month','final_Day','bucketed_Time','final_County'],outputCol = 'final_features'),
  classification.RandomForestClassifier(labelCol='encoded_Fatalaccident', featuresCol='final_features',numTrees = 200,maxDepth = 4)])

In [44]:
#Fitting model1 with training data
fit2 = model1.fit(training)

In [45]:
#Transforming model1 on to validation data
dataframe2 = fit2.transform(validation)

In [46]:
#Prediction vs Actual for model1
display(dataframe1.select('Prediction','Fatal Accident')

Out[ 185 ]: DataFrame[Prediction: double, Fatal Accident: string]

Accuracy is calculated using MultiClass Classification Evaluator and it comes to be 61%

In [48]:
#Accuracy
evaluator = MulticlassClassificationEvaluator(labelCol = 'encoded_Fatalaccident', predictionCol = 'prediction',metricName = 'accuracy')
accuracy2 = evaluator.evaluate(dataframe2)
#Error percentage:
print("Test Error = %g" % (1.0 - accuracy2))
print(accuracy2)

Test Error = 0.382098
0.617902236803

In [49]:
confusion_matrix('Fatal Accident','Prediction')

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-2809427405719332> in <module> () 
 ----> 1 confusion_matrix ( 'Fatal Accident' , 'Prediction' ) 

 /databricks/python/local/lib/python2.7/site-packages/sklearn/metrics/classification.pyc in confusion_matrix (y_true, y_pred, labels, sample_weight) 
 238 
 239 """
 --> 240 y_type , y_true , y_pred = _check_targets ( y_true , y_pred ) 
 241 if y_type not in ( "binary" , "multiclass" ) : 
 242 raise ValueError ( "%s is not supported" % y_type ) 

 /databricks/python/local/lib/python2.7/site-packages/sklearn/metrics/classification.pyc in _check_targets (y_true, y_pred) 
 70 y_pred : array or indicator matrix 
 71 """
 ---> 72 check_consistent_length ( y_true , y_pred ) 
 73 type_true = type_of_target ( y_true ) 
 74 type_pred = type_of_target ( y_pred ) 

 /databricks/python/local/lib/python2.7/site-packages/sklearn/utils/validation.pyc in check_consistent_length (*arrays) 
 179 if len ( uniques ) > 1 : 
 180 raise ValueError("Found input variables with inconsistent numbers of"
 --> 181 " samples: %r" % [int(l) for l in lengths])
 182 
 183 

 ValueError : Found input variables with inconsistent numbers of samples: [14, 10]

MODEL 3: **Gradient Boosting**

The feature Engineering techniques used are:

**StringIndexer**

**OneHotEncoder**

**Bucketizer**(For Time)

All these techniques are pipelined and the Machine Learning models are fitted with the training data and validated on validation data

In [51]:
#Model 3
model2 = Pipeline(stages=[feature.StringIndexer(inputCol='Month', outputCol='encoded_Month'),
                          feature.StringIndexer(inputCol = 'County Name',outputCol = 'encoded_County'),
                          feature.StringIndexer(inputCol = 'Day of Week',outputCol = 'encoded_Day'),
                          feature.StringIndexer(inputCol = 'Fatal Accident',outputCol = 'encoded_Fatalaccident'),
                          feature.StringIndexer(inputCol = 'Time',outputCol = 'encoded_Time'),
                          feature.OneHotEncoder(inputCol = 'encoded_Month',outputCol = 'final_Month'),
                          feature.OneHotEncoder(inputCol = 'encoded_Day',outputCol = 'final_Day'),
                          feature.OneHotEncoder(inputCol = 'encoded_County',outputCol = 'final_County'),
                          feature.Bucketizer(splits=splits, inputCol='encoded_Time', outputCol="bucketed_Time"),
                 feature.VectorAssembler(inputCols = ['final_Month','final_Day','bucketed_Time','final_County'],outputCol = 'final_features'),
                 classification.GBTClassifier(labelCol='encoded_Fatalaccident', featuresCol='final_features',maxIter = 10)])

In [52]:
#Fitting model2 with training data
fit3 = model2.fit(training)

In [53]:
#Transforming model2 on to validation data
dataframe3 = fit3.transform(validation)

In [54]:
#Prediction vs Actual for model2
df3 = dataframe1.select('Prediction','Fatal Accident')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4345105894762168> in <module> () 
 1 #Prediction vs Actual for model2 
 ----> 2 df3 = dataframe1 . select ( 'Prediction' , 'Fatal Accident' ) 

 NameError : name 'dataframe1' is not defined

Accuracy is calculated using MultiClass Classification Evaluator and it is found to be 69%

In [56]:
#Accuracy
evaluator = MulticlassClassificationEvaluator(labelCol = 'encoded_Fatalaccident', predictionCol = 'prediction',metricName = 'accuracy')
accuracy3 = evaluator.evaluate(dataframe3)
#Error percentage:
print("Test Error = %g" % (1.0 - accuracy3))

Test Error = 0.315529

In [57]:
print('LogisticRegression: ',accuracy1)
print('RandomForest: ' ,accuracy2)
print('GradientBoosting: ' , accuracy3)

('LogisticRegression: ', 0.6840410871008598)
('RandomForest: ', 0.6179022368031959)
('GradientBoosting: ', 0.6844707050756295)

**Gradient Boosting** is applied to the final 10% test data since it gave the highest accuracy using Validation Data

In [59]:
#Model3(Gradient Boosting ) will be our final model.
#Applying Gradient Boosting to test data
d4 = fit3.transform(test_data)

Final Accuracy for the test data was calculated using MultiClass Classification Evaluator and it was found to be **68.28%** and the best model is **Gradient Boosting**

In [61]:
#Accuracy
evaluator = MulticlassClassificationEvaluator(labelCol = 'encoded_Fatalaccident', predictionCol = 'prediction',metricName = 'accuracy')
final_accuracy = evaluator.evaluate(d4)
#Error percentage:
print("Test Error = %g" % (1.0 - final_accuracy))
print(final_accuracy)

Test Error = 0.317171
0.682828597494

In [62]:
#Final Accuracy of our model is 68.28% for our test data